In [ ]:
import pickle
import copy
import fiona

import alphashape as ashp
import pandas as pd
import numpy as np
import colorcet as cc
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import LineString, Polygon, Point, MultiPoint
from shapely.ops import transform, unary_union
from shapely import affinity
from descartes import PolygonPatch
from scipy.optimize import minimize
from multiprocessing import Pool
from tqdm import tqdm
from matplotlib import colors
from pyproj import Proj

In [ ]:
plt.style.use('dark_background')
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
SM_H5_PATH = '/home/yang/output/eswe/smooth/'
CLS_H5_PATH = '/home/yang/output/eswe/cls/'
OUTPUT_PATH = '/home/yang/output/eswe/opt/'
EVENT = 'jkwh'
MACHINE_IDS = ['2388', '6088']
DATE = '072019'
FIELD_ID = 'f8'
sm_log_names = ['-'.join([EVENT, mid, DATE, FIELD_ID, 'gps-sm.h5']) for mid in MACHINE_IDS]
cls_log_names = ['-'.join([EVENT, mid, DATE, FIELD_ID, 'gps-cls.h5']) for mid in MACHINE_IDS]
track_colors = ['r', 'royalblue']

In [ ]:
# # Set up dimensions in ft
# w_ft = [30, 32, 35]
# # Convert these to meters
# w_m = [w*0.3048 for w in w_ft]
# # Create dimension combos
# w_comb = [(w0, w1) for w0 in w_m for w1 in w_m]
w_comb = (9.754,9.754)

In [ ]:
# Load the dataframes
gpss = {}
for mid, n1, n2 in zip(MACHINE_IDS, sm_log_names, cls_log_names):
    # Concatenate each machine's smoothed and classified data
    df_sm = pd.read_hdf(SM_H5_PATH + n1)
    df_cls = pd.read_hdf(CLS_H5_PATH + n2)
    df_concat = pd.concat([df_cls, df_sm[['xs', 'ys']]], axis=1)
    # Just like in the previous two steps, we want to group the points because there might be gaps
    # Compute time difference
    gps_seg = df_concat.copy()
    gps_seg['ts_diff'] = gps_seg['ts'].diff()
    gps_seg.loc[0, 'ts_diff'] = 0
    # A trick vector that could be used for grouping
    gps_seg['ts_diff_binary'] = gps_seg['ts_diff'] > 10
    gps_seg['ts_diff_binary_cumsum'] = gps_seg['ts_diff_binary'].apply(lambda x: 1 if x else 0).cumsum()
    # Points groups
    grp = gps_seg.groupby('ts_diff_binary_cumsum')
    # Make indices right
    gps_seg['seg_num'] = grp.ngroup()
    gps_seg.loc[0, 'seg_num'] = 0
    gps_seg = gps_seg.drop(columns=['ts_diff', 'ts_diff_binary', 'ts_diff_binary_cumsum'])
    # Add it to a dict for easy query
    gpss[mid] = gps_seg

In [ ]:
# https://stackoverflow.com/questions/59903415/how-do-i-calculate-the-angle-between-two-points
def get_track(r):
    delta_x = r['xs'] - r['xss']
    delta_y = r['ys'] - r['yss']
    deg_r = np.arctan2(delta_y, delta_x)
    deg_d = deg_r * 180 / np.pi - 90
    deg_true_north = (deg_d + 360) % 360
    return deg_true_north

In [ ]:
d0 = gpss[MACHINE_IDS[0]]
d1 = gpss[MACHINE_IDS[1]]

In [ ]:
d0 = d0.drop(columns=['lat', 'lon', 'alt', 'cv_prob', 'nct_prob'])
d1 = d1.drop(columns=['lat', 'lon', 'alt', 'cv_prob', 'nct_prob'])

In [ ]:
pts_grps0 = [g[1] for g in d0.groupby('seg_num')]
pts_grps1 = [g[1] for g in d1.groupby('seg_num')]

In [ ]:
# Load the boundary
fss = gpd.read_file('/home/yang/data/raw/jkwh.kml', driver='KML')
fss.head()
field_name = FIELD_ID
fs = fss[fss['Name'] == field_name].copy()
fs = fs.reset_index(drop=True)
proj = Proj(proj='utm', zone=13, ellps='WGS84', preserve_units=False)
fs_poly = transform(lambda x, y, z: proj(x, y), fs['geometry'][0])

In [ ]:
# Plot the tracks and field polygons
# %matplotlib notebook
fig, ax = plt.subplots(figsize=(10,6))
d0.plot.scatter(x='xs', y='ys', s=1, c='r', ax=ax)
d1.plot.scatter(x='xs', y='ys', s=1, c='royalblue', ax=ax)
ax.add_patch(PolygonPatch(fs_poly, fc='none', ec='magenta', ls='--'))
ax.axis('equal')

In [ ]:
# From the figure, look at which track corresponds to the outer shape of the field polygon
# OUTER_INDEX = 1
# if OUTER_INDEX == 0:
#     pts = d0.apply(lambda r: Point(r['xs'], r['ys']), axis=1)
#     fs_ashp = ashp.alphashape(pts, 0.01)
# else:
#     pts = d1.apply(lambda r: Point(r['xs'], r['ys']), axis=1)
#     fs_ashp = ashp.alphashape(pts, 0.01)
# # Create field polygon collections
# fs_ashp_f = {}
# fs_ashp_f = Polygon(fs_ashp.exterior.parallel_offset(w_comb[OUTER_INDEX] / 2, side='left', join_style=1))

In [ ]:
def gen_data_chunks(pts_grps, n=500):
    data_chunks = []
    # Go through the pts_grps
    for pts_grp in pts_grps:
        # If the num. of data points is less than 10000, we don't do anything
        if len(pts_grp) < n:
            data_chunks.append(pts_grp)
        else: # Otherwise we will break it apart into n (default: 500) num. row data chunks
            m = 0
            for g, df in pts_grp.groupby(np.arange(len(pts_grp)) // n):
#                 if m > 0:
#                     # We don't want discontinued path so that we will be missing polygons,
#                     # so we will generate length-2 chunks to piece all the chunks together
#                     data_chunks.append(pd.concat([data_chunks[-1].iloc[-1], df.iloc[0]], axis=1).transpose())
                data_chunks.append(df)
                m += 1
    return data_chunks

In [ ]:
def gen_shps(pts_grp, params):
    xoff, yoff, w = params
    # Make an copy of the points data
#     df = pts_grp[pts_grp['mode'] != 'nw-p'].copy()
#     df = df.reset_index(drop=True)
    df = pts_grp.copy()
    # Create a column for *geospatially* shifted points
    df['geom_pt_s'] = np.nan
    # Sanity check on the length of the dataframe
#     print(len(df))
    if len(df) <= 2:
#         print('Only one point in group, this should never happen')
#         print('len <= 1')
        return df
    # Shift coordinates up by one timestep
    df['xs_s'] = df['xs'].shift(-1)
    df['ys_s'] = df['ys'].shift(-1)
    # Make sure every point has a shifted coordinate
    df.loc[df.index[-1], 'xs_s'] = df.loc[df.index[-2], 'xs_s'] + 0.01
    df.loc[df.index[-1], 'ys_s'] = df.loc[df.index[-2], 'ys_s'] + 0.01
    # Create gdf from original points for processing
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['xs'], df['ys']))
    # Rename the geometry to avoid confusion
    gdf.rename_geometry('geom_pt_o', inplace=True)
    # Get the freebies (original points) from the gdf
    df['geom_pt_o'] = gdf['geom_pt_o']
    # Create linestring from original points to time-shifted points
    gdf['geom_ls_o2s'] = gdf.apply(lambda r: LineString([r['geom_pt_o'], Point(r['xs_s'], r['ys_s'])]), axis=1)
#     # HACK: if the linestring is too short, drop these rows
    gdf['geom_ls_o2s_len'] = gdf.apply(lambda r: r['geom_ls_o2s'].length, axis=1)
    gdf.loc[gdf['geom_ls_o2s_len'] < 1e-2, 'geom_ls_o2s_len'] = 1e-2
    # Scale `ls_o2s` to extend it to the right length
    # https://shapely.readthedocs.io/en/latest/manual.html#shapely.affinity.scale
    gdf['geom_ls_o2s_scaled'] = gdf.apply(lambda r: affinity.scale(r['geom_ls_o2s'], \
        xfact=abs(yoff)/r['geom_ls_o2s_len'], yfact=abs(yoff)/r['geom_ls_o2s_len'], origin=r['geom_pt_o']), axis=1)
    # Handle four cases
    if (xoff == 0) & (yoff == 0):
        # Both offsets are 0, no shift geospatially
        df['geom_pt_s'] = df['geom_pt_o']
    if (xoff != 0) & (yoff != 0):
        # Both offsets not 0, need to scale, and then parallel shift
        # Then we need to parallel shift the scaled linestring
        if xoff < 0:
            offset_dir = 'left'
            b_idx = 0
            r_idx = 1
        else:
            offset_dir = 'right'
            b_idx = 1
            r_idx = 0
        gdf['geom_ls_o2s_scaled_po'] = gdf.apply(lambda r: \
            r['geom_ls_o2s_scaled'].parallel_offset(abs(xoff), offset_dir) if r['geom_ls_o2s_scaled'].is_valid else None, axis=1)
        # Depending on the sign of yoff, we need to rotate the parallel shifted linestring
        if yoff > 0:
            rotation_angle = 360
        else:
            rotation_angle = 180
        # Rotate the line
        gdf['geom_ls_o2s_scaled_po_rot'] = gdf.apply(lambda r: affinity.rotate(r['geom_ls_o2s_scaled_po'], \
            rotation_angle, origin=r['geom_ls_o2s_scaled_po'].boundary[b_idx]) if r['geom_ls_o2s_scaled_po'] != None else None, axis=1)
        # Obtain the coordinate
        df['geom_pt_s'] = gdf.apply(lambda r: r['geom_ls_o2s_scaled_po_rot'].boundary[r_idx] if r['geom_ls_o2s_scaled_po_rot'] != None else None, axis=1)
    if (xoff == 0) & (yoff != 0):
        # Depending on the sign of yoff, we need to rotate the parallel shifted linestring
        if yoff > 0:
            rotation_angle = 360
        else:
            rotation_angle = 180
        r_idx = 1
        # Rotate the line
        gdf['geom_ls_o2s_scaled_rot'] = gdf.apply(lambda r: \
            affinity.rotate(r['geom_ls_o2s_scaled'], rotation_angle, origin=r['geom_pt_o']) if r['geom_ls_o2s_scaled'].is_valid else None, axis=1)
        # xoff is 0, so we need to get the right linestring by scaling the linestring, then get the endpoint with no parallel shift
        df['geom_pt_s'] = gdf.apply(lambda r: r['geom_ls_o2s_scaled_rot'].boundary[r_idx] if r['geom_ls_o2s_scaled_rot'] != None else None, axis=1)
    if (xoff != 0) & (yoff == 0):
        # yoff is 0, so we need the get the linestring, no scaling, just parallel shift
        if xoff < 0:
            offset_dir = 'left'
            b_idx = 0
        else:
            offset_dir = 'right'
            b_idx = 1
        gdf['geom_ls_o2s_scaled_po'] = gdf.apply(lambda r: \
            r['geom_ls_o2s'].parallel_offset(abs(xoff), offset_dir) if r['geom_ls_o2s'].is_valid else None, axis=1)
        df['geom_pt_s'] = gdf.apply(lambda r: r['geom_ls_o2s_scaled_po'].boundary[b_idx] if r['geom_ls_o2s_scaled_po'] != None else None, axis=1)
            
    # Let's generate the shapes now
    df_shp = df.copy()
    df_shp = df_shp.dropna(subset=['geom_pt_s']).reset_index(drop=True)
    # Shift the points up by one timestep
    df_shp['geom_pt_s_s'] = df_shp['geom_pt_s'].shift(-1)
    # Create linestrings by connecting the paths
    df_shp = df_shp.iloc[:-2].copy()
    df_shp['geom_ls_path'] = df_shp.apply(lambda r: LineString([r['geom_pt_s'], r['geom_pt_s_s']]) if (r['geom_pt_s'] != None) & (r['geom_pt_s_s'] != None) else None, axis=1)
    # Extend the each path to left/right to obtain header edge points
    geom_ls_path_gs = gpd.GeoSeries(df_shp['geom_ls_path'])
    df_shp['geom_pt_edge_l'] = geom_ls_path_gs.apply(lambda r: r.parallel_offset(w / 2, 'left').boundary[1] if (r.length > 1e-5) else np.nan)
    df_shp['geom_pt_edge_r'] = geom_ls_path_gs.apply(lambda r: r.parallel_offset(w / 2, 'right').boundary[0] if (r.length > 1e-5) else np.nan)
    # Create dataframe to line these points up
    df_edge_pts = pd.concat([df_shp['geom_pt_edge_l'].rename('l'), df_shp['geom_pt_edge_l'].shift(-1).rename('ls'), \
                             df_shp['geom_pt_edge_r'].shift(-1).rename('rs'), df_shp['geom_pt_edge_r'].rename('r')], axis=1)
    # Drop the last one since it will be nan
    df_edge_pts = df_edge_pts.drop(index=[df_edge_pts.index[-1]])
    # Sometimes this happens and we don't have any polygons
    if len(df_edge_pts) == 0:
        return df_shp
    # Create these polygons and swath lines
    df_shp['geom_poly_cvg'] = df_edge_pts.apply(lambda r: Polygon([r['l'], r['ls'], r['rs'], r['r']]).convex_hull \
                                                if (r['l'] != None) & (r['ls'] != None) & (r['rs'] != None) & (r['r'] != None) else None, axis=1)
    df_shp['geom_ls_swath'] = df_edge_pts.apply(lambda r: LineString([r['l'], r['r']]) if (r['l'] != None) & (r['r'] != None) else None, axis=1)
    # Check discontiuities
#     poly_cvg = gpd.GeoSeries(df_shp['geom_poly_cvg'])
#     df_shp = df_shp.drop(index=poly_cvg[poly_cvg.area > 50].index).reset_index(drop=True)
        
    return df_shp

In [ ]:
# [len(p) for p in pts_grps0]
# df_shp = gen_shps(pts_grps0[5], [0,0,w_comb[0]])

In [ ]:
# Obtain coverage polygons and swath linestrings
# p = Pool(15)
# res0 = [p.apply_async(gen_shps, args=(g, [0,0,w_comb[0]])) for g in pts_grps0]
# p.close()
# p.join()
# p.terminate()

In [ ]:
# # Concatenate all the dataframes within the group
# df = pd.concat([r.get() for r in res0])

In [ ]:
# df_shp_o = gen_shps(pts_grps1[2], [0,0,9.144])
# df_shp = gen_shps(pts_grps1[0], [0,8,9.144])

In [ ]:
# %matplotlib notebook
# fig, ax = plt.subplots(figsize=(10,6))
# polys_cvg_o = gpd.GeoSeries(df_shp_o.loc[:, 'geom_poly_cvg'])
# polys_cvg = gpd.GeoSeries(df_shp.loc[:, 'geom_poly_cvg'])
# polys_cvg.plot(alpha=0.8, color='red', ax=ax)
# polys_cvg_o.plot(alpha=0.4, color='royalblue', ax=ax)
# ax.add_patch(PolygonPatch(fs_poly, fc='none', ec='cyan', ls='--'))
# ax.axis('equal')

In [ ]:
def gen_shps_new(pts_grp, params):
    xoff, yoff, w = params
    df = pts_grp.copy()
    df['geom_pt_o'] = np.nan # column for offsetted points
    # If we have only we point in the grp, then just return it with a ring buffer
    if len(df) == 1:
        print('This should never happen, only one point in group')
        df['geom_poly_cvg'] = Point(df['xs'].values, df['ys'].values).buffer(w)
        return df
    # Shift coordinates up by one time step
    df['xss'] = df['xs'].shift(-1)
    df['yss'] = df['ys'].shift(-1)
    # Hack: make sure every point has a coordinate
    df.loc[df.index[-1], 'xss'] = df.loc[df.index[-2], 'xss'] + 0.01
    df.loc[df.index[-1], 'yss'] = df.loc[df.index[-2], 'yss'] + 0.01
    # Create gdf from original points for processing
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['xs'], df['ys']))
    # Rename the geometry to avoid confusion
    gdf.rename_geometry('geom_pt_s', inplace=True)
    # Get the original points
    df['geom_pt_s'] = gdf['geom_pt_s']
    # Put xs, ys shifted into gdf
    gdf['geom_pt_ss'] = gdf.apply(lambda r: Point(r['xss'],r['yss']), axis=1)
    # Create linestring from pt_s to pt_ss
    gdf['geom_ls_s2ss'] = gdf.apply(lambda r: LineString([r['geom_pt_s'], r['geom_pt_ss']]), axis=1)
    # Create point buffer from geom_pt_s
    # xoff is 0 but yoff is not, we simply create the buffer but do not compute the parallel offset
    if (xoff == 0) & (yoff != 0):
        print('xoff == 0, yoff != 0')
        # When xoff is 0, we only need yoff
        # Method: 1) create a circle buffer and use it to intersect with the linestring connecting pt(t) and pt(t+1)
        #         2) if yoff > 0, the endpoint of the intersection is what we want (rotation by 360) doesn't change the coordinate
        #            if yoff < 0, the intersected linestring should be rotated 180 to obtain the endpoint
        if yoff > 0:
            rotation_angle = 360
        else:
            rotation_angle = 180
        gdf['geom_ls_a'] = gdf.apply(lambda r: r['geom_pt_s'].buffer(abs(yoff)).intersection(r['geom_ls_s2ss']), axis=1)
        rotated_ls_a = gpd.GeoSeries(gdf.apply(lambda r: affinity.rotate(r['geom_ls_a'], rotation_angle, origin=r['geom_pt_s']), axis=1))
        df['geom_pt_o'] = rotated_ls_a.apply(lambda r: r.boundary[1])
    elif (xoff != 0) & (yoff == 0):
#         print('xoff != 0, yoff == 0')
        # When yoff is 0, we only need xoff
        # See method when xoff == 0 and yoff != 0
        if xoff > 0:
            rotation_angle = -90
        else:
            rotation_angle = 90
        gdf['geom_ls_a'] = gdf.apply(lambda r: r['geom_pt_s'].buffer(abs(xoff)).intersection(r['geom_ls_s2ss']), axis=1)
        rotated_ls_a = gpd.GeoSeries(gdf.apply(lambda r: affinity.rotate(r['geom_ls_a'], rotation_angle, origin=r['geom_pt_s']), axis=1))
        df['geom_pt_o'] = rotated_ls_a.apply(lambda r: r.boundary[1])
    elif (xoff == 0) & (yoff == 0):
#         print('xoff == 0, yoff == 0')
        df['geom_pt_o'] = df['geom_pt_s']
    else:
#         print('xoff != 0, yoff != 0')
        gdf['geom_ls_a'] = gdf.apply(lambda r: r['geom_pt_s'].buffer(abs(yoff)).intersection(r['geom_ls_s2ss']), axis=1)
        # Get the intersection linestring and rotate it based on yoff > 0 or < 0
        if yoff > 0:
            rotation_angle = 360
            rotated = False
        else:
            rotation_angle = 180
            rotated = True
        rotated_ls_a = gpd.GeoSeries(gdf.apply(lambda r: affinity.rotate(r['geom_ls_a'], rotation_angle, origin=r['geom_pt_s']), axis=1))
        
        if (rotated) & (xoff > 0):
            offset_dir = 'left'
            bidx = 1
        elif (rotated) & (xoff < 0):
            offset_dir = 'right'
            bidx = 0
        elif (~rotated) & (xoff > 0):
            offset_dir = 'right'
            bidx = 0
        elif (~rotated) & (xoff < 0):
            offset_dir = 'left'
            bidx = 1
            
        gdf['geom_ls_a_po'] = rotated_ls_a.apply(lambda r: r.parallel_offset(abs(xoff), offset_dir))
        df['geom_pt_o'] = gdf.apply(lambda r: r['geom_ls_a_po'].boundary[bidx], axis=1)
    
    # Let's generate the shapes now
    df_shp = df.copy()
    df_shp['geom_pt_o_s'] = df_shp['geom_pt_o'].shift(-1)
    # Last point will be null
    if len(df_shp) == 2:
        df_shp = df_shp.iloc[:-1].copy()
    else:
        df_shp = df_shp.iloc[:-2].copy()
    # Check if there is any nan
    nan_idx = df_shp[df_shp['geom_pt_o_s'].isna()].index.values
    if len(nan_idx) != 0:
        # This should not happen since nan's should have been already removed in prior steps
        return -1
    # Create path linestrings
    df_shp['geom_ls_path'] = df_shp.apply(lambda r: LineString([r['geom_pt_o'], r['geom_pt_o_s']]), axis=1)
    # When we only have two points, at this point there will be only one linestring created
    # So we will need to connect pt(t+1) to pt(t) to create a linestring for properly creating the polygon
    if len(df_shp) == 1:
#         print('length is 1')
        df_shp.loc[df_shp.index[0]+1, 'geom_ls_path'] = LineString([df_shp.loc[df_shp.index[0], 'geom_pt_o_s'], df_shp.loc[df_shp.index[0], 'geom_pt_s']])
    
    # Check if any path has distance 0
    ls_path_zero_idx = df_shp[df_shp.apply(lambda r: r['geom_ls_path'].distance == 0, axis=1)].index
    if len(ls_path_zero_idx) != 0:
        # This should not happen either
        return -1
    # Check if any path is None
    ls_path_nan_idx = df_shp[df_shp['geom_ls_path'] == None].index
    if len(ls_path_nan_idx) != 0:
        # This should not happen either
        return -1
    # Create left and right edge points by extending 1/2 swath width
    try:
        geom_ls_path_gs = gpd.GeoSeries(df_shp['geom_ls_path'])
#         df_shp['geom_pt_edge_l'] = df_shp.apply(lambda r: r['geom_ls_path'].parallel_offset(w / 2, 'left').boundary[1], axis=1)
#         df_shp['geom_pt_edge_r'] = df_shp.apply(lambda r: r['geom_ls_path'].parallel_offset(w / 2, 'right').boundary[0], axis=1)
        df_shp['geom_pt_edge_l'] = geom_ls_path_gs.apply(lambda r: r.parallel_offset(w / 2, 'left').boundary[1] if (r.length > 1e-5) else np.nan)
        df_shp['geom_pt_edge_r'] = geom_ls_path_gs.apply(lambda r: r.parallel_offset(w / 2, 'right').boundary[0] if (r.length > 1e-5) else np.nan)
    except IndexError as e:
        print('IndexError!!!! {}, {}, {}, {}, {}'.format(xoff, yoff, w, len(df_shp), sum(df_shp['geom_ls_path'] == None)))
        return df_shp
    # Create dataframe to line these points up
    df_edge_pts = pd.concat([df_shp['geom_pt_edge_l'].rename('l'), df_shp['geom_pt_edge_l'].shift(-1).rename('ls'), \
                             df_shp['geom_pt_edge_r'].shift(-1).rename('rs'), df_shp['geom_pt_edge_r'].rename('r')], axis=1)
    
    # Drop the last one since it will be nan
    df_edge_pts = df_edge_pts.drop(index=[df_edge_pts.index[-1]])
    # Drop any nan columns
    df_edge_pts = df_edge_pts.dropna()
    # Create these polygons and swath lines
    df_shp['geom_poly_cvg'] = gpd.GeoSeries(df_edge_pts.apply(lambda r: Polygon([r['l'], r['ls'], r['rs'], r['r']]).convex_hull, axis=1))
    df_shp['geom_ls_swath'] = gpd.GeoSeries(df_edge_pts.apply(lambda r: LineString([r['l'], r['r']]), axis=1))
    
    # Drop the last row when creating polygon using only 2 points
    if len(df_shp) == 2:
        df_shp = df_shp.drop(index=[df_shp.index[-1]])
    
    return df_shp

In [ ]:
def f(X_to_opt, args):
    # Obtain the arguments
    xoff0, yoff0, xoff1, yoff1 = X_to_opt
    p, w0, w1, m0_grps, m1_grps = args
    # Obtain the async results
    res0 = [p.apply_async(gen_shps, args=(g, [xoff0,yoff0,w0])) for g in m0_grps]
    res1 = [p.apply_async(gen_shps, args=(g, [xoff1,yoff1,w1])) for g in m1_grps]
    # Concatenate all the dataframes within the group
    df_shp0 = pd.concat([r.get() for r in res0])
    df_shp1 = pd.concat([r.get() for r in res1])
    # Get only the working polygons
#     poly_cvg0 = gpd.GeoSeries(df_shp0.loc[df_shp0['mode'] == 'w', 'geom_poly_cvg'])
#     poly_cvg1 = gpd.GeoSeries(df_shp1.loc[df_shp1['mode'] == 'w', 'geom_poly_cvg'])
    poly_cvg0 = gpd.GeoSeries(df_shp0.loc[:, 'geom_poly_cvg'])
    poly_cvg1 = gpd.GeoSeries(df_shp1.loc[:, 'geom_poly_cvg'])
    # Compute the total covered area with overlap
    area_covered_w_overlap = poly_cvg0.area.sum() + poly_cvg1.area.sum()
    # Obtain the covered area polygon
    poly_cvg_union = unary_union([poly_cvg0.unary_union, poly_cvg1.unary_union])
    poly_cvg_actual = fs_poly.intersection(poly_cvg_union)
#     poly_cvg_actual = fs_ashp_f.intersection(poly_cvg_union)
    # Get the covered area
    area_covered_actual = poly_cvg_union.area
    # Compute the overlap area
    area_overlap = area_covered_w_overlap - area_covered_actual
    # Get the skipped area
    poly_skip = fs_poly.difference(poly_cvg_actual)
#     poly_skip = fs_ashp_f.difference(poly_cvg_actual)
    print('xoff{}: {:.2f} | yoff{}: {:.2f} | xoff{}: {:.2f} | yoff{}: {:.2f} | w{}: {:.2f} | w{}: {:.2f}\na_o: {:.2f} | a_s: {:.2f} | a_to: {:.2f} | a_a: {:.2f} | a_fs: {:.2f}'.format(\
        MACHINE_IDS[0], xoff0, MACHINE_IDS[0], yoff0, MACHINE_IDS[1], xoff1, MACHINE_IDS[1], yoff1, \
        MACHINE_IDS[0], w0, MACHINE_IDS[1], w1, area_overlap, poly_skip.area, area_covered_w_overlap, area_covered_actual, fs_poly.area))
    print('\tcost: {:.2f}'.format(1/2*area_overlap+1/2*poly_skip.area))
    return 1/2*area_overlap+1/2*poly_skip.area
#     print('\tcost: {:.2f}'.format(-area_covered_actual))
#     return poly_skip.area

In [ ]:
# Before optimization, let's do a few test runs
# p = Pool(15)
# xoff0 = 0
# yoff0 = 0
# xoff1 = 0
# yoff1 = 0
# for w_tuple in w_comb:
#     print(w_tuple)
#     res = f([xoff0,yoff0,xoff1,yoff1], [p,w_tuple[0],w_tuple[1],pts_grps0,pts_grps1])
# p.close()
# p.join()
# p.terminate()

# # Get results for both machines
# res0 = [p.apply_async(gen_shps_new, args=(g, [xoff0,yoff0,w0])) for g in pts_grps0]
# res1 = [p.apply_async(gen_shps_new, args=(g, [xoff1,yoff1,w1])) for g in pts_grps1]

# # Concatenate all the dataframes within the group
# df_shp0 = pd.concat([r.get() for r in res0])
# df_shp1 = pd.concat([r.get() for r in res1])
# fig, ax = plt.subplots(figsize=(10,6))
# shp_cvg0 = gpd.GeoSeries(df_shp0.loc[:, 'geom_poly_cvg'])
# shp_cvg1 = gpd.GeoSeries(df_shp1.loc[:, 'geom_poly_cvg'])
# shp_cvg0.plot(alpha=0.6, color='r', ax=ax)
# shp_cvg1.plot(alpha=0.6, color='blue', ax=ax)
# ax.add_patch(PolygonPatch(fs_poly, fc='none', ec='orange', ls='--'))

In [ ]:
pts_grps0_chunks = gen_data_chunks(pts_grps0)
pts_grps1_chunks = gen_data_chunks(pts_grps1)

In [ ]:
# for idx, c in enumerate(pts_grps1_chunks):
#     print(idx, len(c))

In [ ]:
# p = Pool(15)
# Get results for both machines
# re = [-137234.58902515998, 0.15708930613405675, 3.1440586726632014, -0.1985936886748788, 8.47810408120722, 9.144, 9.144]
# res0 = [p.apply_async(gen_shps, args=(g, [re[1],re[2],9.144])) for g in pts_grps0_chunks]
# res1 = [p.apply_async(gen_shps, args=(g, [0,0,9.144])) for g in pts_grps1_chunks]
# p.close()
# p.join()
# p.terminate()

# # Concatenate all the dataframes within the group
# df_shp0 = pd.concat([r.get() for r in res0])
# df_shp1 = pd.concat([r.get() for r in res1])
# fig, ax = plt.subplots(figsize=(10,6))
# shp_cvg0 = gpd.GeoSeries(df_shp0.loc[:, 'geom_poly_cvg'])
# shp_cvg1 = gpd.GeoSeries(df_shp1.loc[:, 'geom_poly_cvg'])
# shp_cvg0.plot(alpha=0.6, color='r', ax=ax)
# shp_cvg1.plot(alpha=0.6, color='royalblue', ax=ax)
# ax.add_patch(PolygonPatch(fs_poly, fc='none', ec='orange', ls='--'))
# ax.axis('equal')

In [ ]:
# p = Pool(15)
# Get results for both machines
# res0 = [p.apply_async(gen_shps, args=(g, [0,0,9.144])) for g in pts_grps0_chunks]
# p.close()
# p.join()
# p.terminate()
# Concatenate all the dataframes within the group
# df_shp0 = pd.concat([r.get() for r in res0])
# df_shp1 = pd.concat([r.get() for r in res1])
# fig, ax = plt.subplots(figsize=(10,6))
# shp_cvg0 = gpd.GeoSeries(df_shp0.loc[:, 'geom_poly_cvg'])
# shp_cvg1 = gpd.GeoSeries(df_shp1.loc[:, 'geom_poly_cvg'])
# shp_cvg0.plot(alpha=0.6, color='r', ax=ax)
# shp_cvg1.plot(alpha=0.6, color='royalblue', ax=ax)
# ax.add_patch(PolygonPatch(fs_poly, fc='none', ec='orange', ls='--'))
# ax.axis('equal')

In [ ]:
# Run optimizer
p = Pool(15)
r = minimize(f, [0,0,0,0], method='Powell', options={'xtol': 1e-2, 'ftol': 1}, \
             bounds=((-10,10),(-10,10),(-10,10),(-10,10)), args=[p,w_comb[0],w_comb[1],pts_grps0_chunks,pts_grps1_chunks])
p.close()
p.join()
p.terminate()

In [ ]:
results = []
results += list(r.x) + list(w_comb)

In [ ]:
print(results)

In [ ]:
# Before optimization, let's do a few test runs
# p = Pool(15)
# # Get results for both machines
# res0 = [p.apply_async(gen_shps, args=(g, [0,0,w_comb[0]])) for g in pts_grps0]
# res1 = [p.apply_async(gen_shps, args=(g, [0,0,w_comb[1]])) for g in pts_grps1]
# # Get shifted results for both machines
# res0_s = [p.apply_async(gen_shps, args=(g, [results[0],results[1],w_comb[0]])) for g in pts_grps0]
# res1_s = [p.apply_async(gen_shps, args=(g, [results[2],results[3],w_comb[1]])) for g in pts_grps1]
# # Close the pool
# p.close()
# p.join()
# p.terminate()

In [ ]:
# # Concatenate all the dataframes within the group
# df_shp0 = pd.concat([r.get() for r in res0])
# df_shp1 = pd.concat([r.get() for r in res1])
# shp_cvg0 = gpd.GeoSeries(df_shp0.loc[:, 'geom_poly_cvg'])
# shp_cvg1 = gpd.GeoSeries(df_shp1.loc[:, 'geom_poly_cvg'])
# df_shp0_s = pd.concat([r.get() for r in res0_s])
# df_shp1_s = pd.concat([r.get() for r in res1_s])
# shp_cvg0_s = gpd.GeoSeries(df_shp0_s.loc[:, 'geom_poly_cvg'])
# shp_cvg1_s = gpd.GeoSeries(df_shp1_s.loc[:, 'geom_poly_cvg'])

# # %matplotlib notebook
# fig, ax = plt.subplots(figsize=(10,6))
# shp_cvg0.plot(alpha=0.6, color='r', ax=ax)
# shp_cvg1.plot(alpha=0.6, color='r', ax=ax)
# shp_cvg0_s.plot(alpha=0.6, color='royalblue', ax=ax)
# shp_cvg1_s.plot(alpha=0.6, color='royalblue', ax=ax)
# ax.add_patch(PolygonPatch(fs_poly, fc='none', ec='orange', ls='--'))
# ax.axis('equal')

In [ ]:
# pickle_name = '-'.join([EVENT, '-'.join(MACHINE_IDS), DATE, FIELD_ID, 'opt-res.pickle'])
# print(pickle_name)
# with open(OUTPUT_PATH + pickle_name, 'wb') as fh:
#     pickle.dump(results, fh, protocol=pickle.HIGHEST_PROTOCOL)